In [29]:
import pandas as pd 
import numpy as np
import seaborn as sns
pd.set_option('max_columns',200)
pd.set_option('max_rows',200)

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
#记住这里是在做regression，所以import的是randomforestregressor

from sklearn.metrics import mean_squared_error
from math import sqrt

### import data

In [30]:
data_train = pd.read_csv('datatrain_final.csv')
data_test = pd.read_csv('datatest_final.csv')
data_train.head()

,channelGrouping.Direct,channelGrouping.Organic.Search,channelGrouping.Referral,visitNumber,device.operatingSystem.Macintosh,device.operatingSystem.Windows,device.deviceCategory.desktop,geoNetwork.continent.Americas,totals.hits,totals.pageviews,totals.bounces,day.Friday,day.Monday,day.Saturday,day.Sunday,day.Thursday,day.Tuesday,day.Wednesday,logTransaction
0,0,1,0,1,0,0,0,1,10,8,0,0,0,0,1,0,0,0,0.0
1,0,0,0,1,1,0,1,0,2,2,0,0,1,0,0,0,0,0,0.0
2,1,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,0.0
3,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0.0
4,0,1,0,1,1,0,1,0,1,1,1,0,0,0,1,0,0,0,0.0


#### X, y _ train, test

In [31]:
X_train = data_train.drop(labels=['logTransaction'],axis=1)

feature_list = list(X_train.columns)

X_train = X_train.values
y_train=data_train['logTransaction'].values 
print(X_train.shape,y_train.shape)

(317942, 18) (317942,)


In [32]:
X_test = data_test.drop(labels=['logTransaction'],axis=1)
X_test = X_test.values

y_test=data_test['logTransaction'].values 
y_test.shape

print(X_test.shape,y_test.shape)

(71460, 18) (71460,)


### parameter

#### rf default parameter

In [33]:
rf = RandomForestRegressor(random_state = 42)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [34]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(55, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [55, 60, 66, 71, 77, 82, 88, 93, 99, 104, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}


In [35]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 42)
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_root_mean_squared_error', 
                              cv = 5, verbose=3, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train, y_train);

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 145.2min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 306.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 552.5min finished


In [36]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

### Evaluate the Best Random Search Model

In [39]:
best_random = rf_random.best_estimator_

best_random.fit(X_train, y_train)

prediction_test = best_random.predict(X_test)

RMSE = rmse_test = sqrt(mean_squared_error(y_test, prediction_test))

print('rmse - test: ',rmse_test)

rmse - test:  1.8439205608416205


In [40]:
from sklearn.metrics import mean_absolute_error

In [42]:
# predict the result for test
mae_test=mean_absolute_error(y_test,prediction_test)
print('mae: ',mae_test)

mae:  0.3150414100179294
